<a href="https://colab.research.google.com/github/akamrume328/tennisvision/blob/feature%2F%231/notebooks/2_model_training_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google ColabでのYOLOv8モデルのトレーニング (GCS対応版)

このノートブックは、Google Cloud Storage (GCS) に保存されたカスタムデータセットを使用して、Google Colab上でYOLOv8モデルをトレーニングするためのものです。
以下の内容をカバーします：
1. Google Cloud Storage (GCS) への認証とgcsfuseのインストール。
2. `ultralytics`ライブラリのインストール。
3. **データセットの準備とGoogle Cloud Storage (GCS) へのアップロード**:
   - YOLOv8形式のデータセット（画像ファイル群、対応するラベルファイル群、そしてデータセット設定ファイル `data.yaml`）を準備します。
   - `data.yaml` には、クラス名、クラス数、そして訓練データと検証データへのパスを記述します。これらのパスは相対パスで記述し、gcsfuseマウント後にローカルファイルのようにアクセスできます。
   - 準備したデータセット全体（画像、ラベル、`data.yaml`等）をGCSバケットにアップロードします。
4. gcsfuseによるGCSバケットのマウントとパス設定。
5. YOLOv8モデルのトレーニング。
6. トレーニング済みモデルと結果のGCS上の保存場所に関する情報。

In [ ]:
# Google Cloud SDKへの認証
from google.colab import auth
auth.authenticate_user()

# (オプション) Google CloudプロジェクトIDを設定
# project_id = "your-gcp-project-id" # ★★★ あなたのGCPプロジェクトIDに置き換えてください ★★★
# !gcloud config set project {project_id}
# print(f"Google CloudプロジェクトIDが {project_id} に設定されました。")
print("Google Cloud SDKへの認証が完了しました。")

Mounted at /content/drive


In [ ]:
# gcsfuseのインストールとセットアップ
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get update
!sudo apt-get install gcsfuse -y

print("gcsfuseのインストールが完了しました。")

In [ ]:
# ultralyticsライブラリをインストール
!pip install ultralytics
print("ultralyticsライブラリのインストールが完了しました。")

## Gitのセットアップ (オプション)

Colabから直接Gitリポジトリにコミットやプッシュを行いたい場合、以下のセルでGitのユーザー名とメールアドレスを設定します。
これは、コミット履歴に記録される作成者情報となります。

**注意:** ここで設定するメールアドレスやユーザー名は公開される可能性があるため、取り扱いに注意してください。

In [ ]:
# Gitのユーザー名とメールアドレスを設定
# 以下の "Your Name" と "youremail@example.com" をご自身のものに置き換えてください。
!git config --global user.name "akamrume328" # ★★★ あなたの名前に置き換えてください ★★★
!git config --global user.email "akamarume@icloud.com" # ★★★ あなたのメールアドレスに置き換えてください ★★★

print("Gitのユーザー名とメールアドレスが設定されました。")
!git config --global --list # 設定内容の確認

Gitのユーザー名とメールアドレスが設定されました。
user.name=akamrume328
user.email=akamarume@icloud.com


## パスの設定

データセットの`data.yaml`ファイルと、トレーニング結果（例: モデルの重み、ログ）を保存するディレクトリのパスを定義します。

## GCSバケットのマウント

gcsfuseを使用してGCSバケットをColabのローカルファイルシステムにマウントします。
これにより、GCS上のファイルにローカルファイルのようにアクセスできるようになります。

**注意:**
- マウントするGCSバケット名 (`bucket_name`) をご自身の環境に合わせて設定してください。
- マウントポイント (`mount_point`) は通常 `/content/gcs` などのディレクトリを使用します。
- Colabのセッションが終了すると、マウントは解除されます。再度ノートブックを実行する際には、このセルも再実行する必要があります。
- マウント後は、`/content/gcs/path/to/file` のようなローカルパスでGCS上のファイルにアクセスできます。

In [ ]:
import os
import subprocess

# --- ★★★ ユーザー設定項目 ★★★ ---
# マウントするGCSバケット名 (gs:// プレフィックスは不要)
bucket_name = 'tennis_vision_dataset'  # ★★★ あなたのGCSバケット名に置き換えてください ★★★

# GCSバケットをマウントするローカルディレクトリ
mount_point = '/content/gcs'  # ★★★ 必要であれば変更してください ★★★

# --- ★★★ 設定項目終了 ★★★ ---

# マウントポイントディレクトリを作成
if not os.path.exists(mount_point):
    os.makedirs(mount_point)
    print(f"マウントポイントディレクトリを作成しました: {mount_point}")
else:
    print(f"マウントポイントディレクトリは既に存在します: {mount_point}")

# 既にマウントされているかチェック
def is_mounted(mount_point):
    result = subprocess.run(['mountpoint', '-q', mount_point])
    return result.returncode == 0

if is_mounted(mount_point):
    print(f"警告: {mount_point} は既にマウントされています。")
    print("既存のマウントをアンマウントしています...")
    subprocess.run(['sudo', 'umount', mount_point])

# gcsfuseを使用してGCSバケットをマウント
print(f"GCSバケット '{bucket_name}' を {mount_point} にマウントしています...")
mount_command = ['gcsfuse', '--implicit-dirs', bucket_name, mount_point]
mount_process = subprocess.run(mount_command, capture_output=True, text=True)

if mount_process.returncode == 0:
    print(f"GCSバケットのマウントが完了しました: gs://{bucket_name} -> {mount_point}")
    
    # マウント成功の確認
    print(f"\nマウントポイント ({mount_point}) の内容:")
    try:
        items = os.listdir(mount_point)
        if items:
            for item in items[:10]:  # 最初の10項目のみ表示
                print(f"- {item}")
            if len(items) > 10:
                print(f"... (他に {len(items) - 10} 個のアイテムがあります)")
        else:
            print("(バケットは空か、アクセス権限がありません)")
    except Exception as e:
        print(f"マウントポイントの内容を確認できませんでした: {e}")
    
    print(f"\n重要: 次の「パスの設定」セルで、パスを {mount_point} 以下のローカルパスに設定してください。")
    print(f"例: dataset_yaml_path = '{mount_point}/datasets/data.yaml'")
    print(f"例: project_output_dir = '{mount_point}/models/YOLOv8_Training_Outputs'")
    
else:
    print(f"エラー: GCSバケットのマウントに失敗しました。")
    print(f"エラー出力: {mount_process.stderr}")
    print("バケット名やアクセス権限を確認してください。")
    print(f"バケット '{bucket_name}' が存在し、適切なアクセス権限があることを確認してください。")

In [ ]:
import os

# --- 重要: これらのパスをマウントしたGCSバケット内の構造に合わせて設定してください ---

# データセットのdata.yamlファイルへのパス (マウントポイント以下のローカルパス)
# 例: '/content/gcs/datasets/data.yaml'
dataset_yaml_path = '/content/gcs/final_merged_dataset/data.yaml'  # ★★★ ここをマウントされたGCS内のdata.yamlへの実際のパスに修正してください ★★★

# トレーニング結果（重み、ログなど）を保存するディレクトリ (マウントポイント以下のローカルパス)
# 例: '/content/gcs/models/YOLOv8_Training_Outputs'
project_output_dir = '/content/gcs/models/YOLOv8_Training_Outputs'  # ★★★ ここをマウントされたGCS内のパスに修正してください ★★★

# この特定のトレーニング実行の名前（project_output_dirのサブディレクトリとして作成されます）
experiment_name = 'tennis_detection_gcs_run1'

# --- トレーニングデバイスの設定 ---
# 'cpu' または 'cuda:0' (または単に 0 や 'gpu') を指定
training_device = 'cuda:0' # ★★★ 'cpu' または 'cuda:0' に変更 ★★★

# --- チェックポイントと再開設定 ---
# トレーニングを再開する場合、ここにチェックポイントファイル (.pt) のローカルパス（マウントポイント以下）を指定します。
# 例: '/content/gcs/models/YOLOv8_Training_Outputs/tennis_detection_gcs_run1/weights/last.pt'
# 新規トレーニングの場合は None または空文字列のままにします。
checkpoint_to_resume = None  # ★★★ 再開する場合、ここにチェックポイントのローカルパスを設定 ★★★

# 何エポックごとにチェックポイントを保存するか (例: 10エポックごと)
save_every_n_epochs = 10  # ★★★ 必要に応じて変更 ★★★

# --- パス設定終了 ---

# プロジェクト出力ディレクトリが存在しない場合は作成
os.makedirs(project_output_dir, exist_ok=True)

print(f"データセットのYAMLパス: {dataset_yaml_path}")
print(f"プロジェクト出力ディレクトリ: {project_output_dir}")
print(f"実験名: {experiment_name}")
print(f"トレーニングデバイス: {training_device}")

if checkpoint_to_resume:
    if os.path.exists(checkpoint_to_resume):
        print(f"トレーニング再開用チェックポイント: {checkpoint_to_resume}")
    else:
        print(f"警告: 指定された再開用チェックポイントが見つかりません: {checkpoint_to_resume}")
        print("新規トレーニングとして開始されます。")
        checkpoint_to_resume = None
else:
    print("新規トレーニングとして開始します。")
print(f"チェックポイント保存頻度: {save_every_n_epochs} エポックごと (0以下なら無効)")

# データセットのYAMLファイルの存在を確認
if not os.path.exists(dataset_yaml_path):
    print(f"エラー: データセットのYAMLファイルが見つかりません: {dataset_yaml_path}")
    print("パスを再確認してください。GCSバケットが正しくマウントされ、ファイルが存在することを確認してください。")
else:
    print("データセットのYAMLファイルが見つかりました。")
    
    # data.yamlの内容を確認
    try:
        with open(dataset_yaml_path, 'r') as f:
            yaml_content = f.read()
        print("\ndata.yamlの内容:")
        print(yaml_content[:500] + "..." if len(yaml_content) > 500 else yaml_content)
        
        # データセットディレクトリの構造を確認
        dataset_dir = os.path.dirname(dataset_yaml_path)
        print(f"\nデータセットディレクトリ ({dataset_dir}) の構造:")
        for root, dirs, files in os.walk(dataset_dir):
            level = root.replace(dataset_dir, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files[:5]:  # 最初の5ファイルのみ表示
                print(f"{subindent}{file}")
            if len(files) > 5:
                print(f"{subindent}... (他に {len(files) - 5} 個のファイル)")
    except Exception as e:
        print(f"data.yamlまたはデータセット構造の確認中にエラーが発生しました: {e}")

## YOLOv8モデルの初期化とトレーニング

### モデルのロードとトレーニング実行

このセルでは、YOLOv8モデルを初期化し、トレーニングを実行します。

**チェックポイントからの再開:**
前の「パスの設定」セルで `checkpoint_to_resume` に有効なチェックポイントファイル（例: `last.pt` や特定の `epoch_XXX.pt`）のパスが指定されている場合、モデルはそのチェックポイントからロードされ、トレーニングが再開されます。これにより、中断したトレーニングを続けたり、既存のモデルをさらにファインチューニングしたりできます。
`checkpoint_to_resume` が `None` であるか、指定されたパスにファイルが存在しない場合は、`model_name` で指定された新しい事前学習済みモデル（例: `yolov8s.pt`）からトレーニングが開始されます。

**定期的なチェックポイント保存:**
`model.train()` メソッドの `save_period` パラメータに、前のセルで設定した `save_every_n_epochs` の値が渡されます。
`save_every_n_epochs` が1以上の場合、指定されたエポック数ごとにモデルの重みが `project_output_dir/experiment_name/weights/epoch_XXX.pt` という名前で保存されます。これにより、トレーニングの途中経過を細かく保存し、必要に応じて特定のエポックから再開できるようになります。

**その他のパラメータ:**
-   `data`: データセット設定ファイル (`data.yaml`) へのパス。
-   `epochs`: 総トレーニングエポック数。
-   `imgsz`: 入力画像のサイズ。
-   `batch`: バッチサイズ。
-   `project`: トレーニング結果が保存される親ディレクトリ。
-   `name`: 今回のトレーニング実行固有のサブディレクトリ名。
-   `exist_ok=True`: `project/name` ディレクトリが既に存在する場合でもエラーとせず、上書きまたは追記を許可します。
-   `device`: トレーニングに使用するデバイス（例: `'cpu'`, `'cuda:0'`）。前のセルで設定した `training_device` 変数の値が使用されます。

トレーニングが開始されると、進行状況、損失、評価メトリクスなどが表示されます。
**トレーニングが正常に完了した場合、このセルの処理がすべて終了すると、Colabのランタイムは自動的に切断され、リソースが解放されます。エラーが発生した場合は、ランタイムは切断されません。**

In [ ]:
from ultralytics import YOLO
import os
from google.colab import runtime
import torch

# GPU メモリクリア関数を定義
def clear_gpu_memory():
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("GPUメモリキャッシュをクリアしました。")
        else:
            print("CUDAが利用できないため、GPUメモリクリアはスキップされました。")
    except Exception as e:
        print(f"GPUメモリクリア中にエラーが発生しました: {e}")

# 事前学習済みのYOLOv8モデルをロードするか、チェックポイントから再開
# 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt' などのサイズを選択可能
# 初めての場合は 'yolov8n.pt' (Nano) または 'yolov8s.pt' (Small) を推奨
model_name = 'yolov8x.pt' # 新規トレーニングの場合のデフォルトモデル

model_load_path = model_name # デフォルトは新規モデル
if checkpoint_to_resume and os.path.exists(checkpoint_to_resume):
    print(f"チェックポイントからトレーニングを再開します: {checkpoint_to_resume}")
    model_load_path = checkpoint_to_resume
elif checkpoint_to_resume:
    print(f"指定されたチェックポイント '{checkpoint_to_resume}' が見つからなかったため、新しい事前学習済みモデル ({model_name}) からトレーニングを開始します。")
else:
    print(f"新しい事前学習済みモデル ({model_name}) からトレーニングを開始します。")

model = YOLO(model_load_path)

# トレーニングのハイパーパラメータを定義
num_epochs = 150  # トレーニングエポック数 (例: 50, 100, 200)
batch_size = 16   # バッチサイズ (GPU/CPUメモリに応じて調整: 例 8, 16, 32)
img_size = 1920    # 入力画像サイズ (例: 640, 1280)

# A100 向け追加チューニング
workers   = 8         # DataLoaderの並列数
lr0       = 0.001     # 初期学習率
optimizer = 'AdamW'   # オプティマイザ
amp       = True      # 混合精度トレーニングを有効化 (FP16)


print(f"モデルのトレーニングを開始します。")
print(f"使用デバイス: {training_device}")
print(f"エポック数: {num_epochs}, バッチサイズ: {batch_size}, 画像サイズ: {img_size}")
if save_every_n_epochs > 0:
    print(f"チェックポイントは {save_every_n_epochs} エポックごとに保存されます。")

print("\n重要: トレーニングが正常に完了した場合、このセルの最後にランタイムが自動的に切断されます。")
print("エラーが発生した場合は、ランタイムは切断されず、エラーメッセージを確認できます。")
print(f"トレーニング結果はマウントされたGCSディレクトリ ({project_output_dir}) に保存される設定になっています。")

# トレーニングを開始
training_successful = False
try:
    print("=== トレーニング開始前のGPUメモリクリア ===")
    clear_gpu_memory()

    results = model.train(
        data=dataset_yaml_path,
        epochs=num_epochs,
        imgsz=img_size,
        batch=batch_size,
        project=project_output_dir, # すべての実行結果を保存するマウントされたGCS上のメインディレクトリ
        name=experiment_name,       # この特定の実行のサブディレクトリ
        exist_ok=True,              # experiment_nameディレクトリが既に存在する場合は上書きを許可
        save_period=save_every_n_epochs if save_every_n_epochs > 0 else -1, # 指定エポックごとに保存 (-1で無効)
        device=training_device,
        workers=workers,                # データローダーのワーカー数 (CPUコア数やデバイスに応じて調整)
        # patience=30,              # 早期終了の忍耐値 (例: 30エポック後に改善がない場合に停止)
        lr0=lr0,                 # 初期学習率
        optimizer= optimizer,
        amp= amp、、        # オプティマイザ (例: 'SGD', 'Adam', 'AdamW')
        cache=True
    )
    print("\nトレーニングが正常に完了しました！")
    full_output_path = os.path.join(project_output_dir, experiment_name)
    print(f"結果、ログ、モデルの重みはマウントされたGCS上の次の場所に保存されました: {full_output_path}")
    training_successful = True

    print("\nトレーニングが正常に完了したため、ランタイムを切断します...")
    print(f"必要なファイルがGCS ({full_output_path}) に保存されていることを確認してください。")
    print("ランタイム切断中...")
    runtime.unassign()
    print("ランタイムが切断されました。Colabの接続は解除されます。")

except torch.cuda.OutOfMemoryError as e:
    print("GPU OOM エラーが発生しました:", e)
    print("GPUメモリをクリアします...")
    clear_gpu_memory()
    print("バッチサイズを小さくするか、画像サイズを下げることを検討してください。")
except Exception as e:
    print(f"トレーニング中に予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()
finally:
    if not training_successful:
        print("\nこのセルの処理は終了しましたが、トレーニング中にエラーが発生したか、正常に完了しませんでした。")
        print(f"エラーが発生した場合でも、部分的な結果がGCS ({os.path.join(project_output_dir, experiment_name)}) に保存されている可能性があります。")

## トレーニング結果と保存されたモデル

トレーニング後、結果（メトリクス、混同行列、モデルの重みなど）は指定したマウントされたGCSディレクトリに保存されます：
`{project_output_dir}/{experiment_name}` (例: `/content/gcs/models/YOLOv8_Training_Outputs/tennis_detection_gcs_run1`)

確認すべき主なファイル：
- **重み:** `weights`サブディレクトリ内 (例: `best.pt`, `last.pt`)
  - `best.pt`: 最良の検証メトリクス（通常はmAP50-95）を達成したモデルの重み。このモデルを推論に使用するのが一般的です。
  - `last.pt`: トレーニングの最終エポックのモデルの重み。
  - `epoch_XXX.pt` (例: `epoch_10.pt`, `epoch_20.pt`): `save_period` で指定されたエポックごとに保存されるチェックポイント。これらを使用して特定のエポックからトレーニングを再開したり、その時点でのモデル性能を評価したりできます。
- **結果CSV:** `results.csv`にはエポックごとのメトリクスの概要が含まれています。
- **プロット:** 混同行列、P-R曲線などのさまざまなプロット (例: `confusion_matrix.png`, `PR_curve.png`)

これらのファイルは自動的にGCSに同期されるため、マウントが解除された後でもGCS上で利用できます。

In [ ]:
import os

# 実験の重みディレクトリ (マウントされたローカルパス)
weights_dir = os.path.join(project_output_dir, experiment_name, 'weights')

print(f"\n重みディレクトリ内のファイルをリストアップします ({weights_dir}):")

if os.path.exists(weights_dir):
    try:
        files_in_weights_dir = os.listdir(weights_dir)
        if files_in_weights_dir:
            for f_name in files_in_weights_dir:
                print(f"- {f_name}")
        else:
            print(f"重みディレクトリ ({weights_dir}) は空です。")
            print("トレーニングが完了していないか、エラーが発生した可能性があります。")
    except Exception as e:
        print(f"重みディレクトリの内容を読み取れませんでした: {e}")
else:
    print(f"\n重みディレクトリが見つかりません: {weights_dir}")
    print("トレーニングが完了していないか、エラーが発生した可能性があります。")

# トレーニング済みモデルの最良の重みへのローカルパス
best_model_path = os.path.join(weights_dir, 'best.pt')
print(f"\n最良のモデルへのローカルパス: {best_model_path}")

if os.path.exists(best_model_path):
    print("最良のモデルファイル (best.pt) が見つかりました。")
    # ファイルサイズも表示
    try:
        file_size = os.path.getsize(best_model_path)
        print(f"ファイルサイズ: {file_size / (1024*1024):.1f} MB")
    except Exception as e:
        print(f"ファイルサイズを取得できませんでした: {e}")
else:
    print("最良のモデルファイル (best.pt) が見つかりません。トレーニングが失敗したか、生成されなかった可能性があります。")

print(f"\n注意: ファイルは自動的にGCSバケット (gs://{bucket_name}) に同期されています。")

## 推奨ハイパーパラメーター設定

YOLOv8xで8400フレーム（imgsz=1920）を学習させる際の目安：

- エポック数: **30～50**  
  - 少ないデータでは早期停止(Early Stopping)を有効にし、学習曲線を監視  
- バッチサイズ(batch): **8～16**  
  - A100(40GB)：batch=16 まで可能。OOM時は batch=8 に  
- 学習率(lr0): **0.01**  
- オプティマイザ: **AdamW**  
- ワーカー数(workers): **8**  
- 画像サイズ(imgsz): **1920**  
- 早期停止(patience): **10** エポック  